#Stable Diffusion LoRA Fine-Tuning


##Setup

###Installing and importing all necessary libraries and modules

In [1]:
!pip install -qq transformers peft accelerate torch torchvision safetensors datasets wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import os
from datasets import load_dataset
from PIL import Image
import wandb

##Dataset preparation

###Preparing a small dataset of 4 images (loaded from an existing Hugging Face dataset).

In [2]:
# Step 1: Load dataset
dataset = load_dataset("sandeshrajx/tom-hiddleston", split="train[:4]")

# Step 2: Save images to directory
os.makedirs("training_data/images", exist_ok=True)

for i, item in enumerate(dataset):
    image: Image.Image = item["image"]
    filename = f"image_{i}.jpg"
    image.save(f"training_data/images/{filename}")


2021-004-GQ-British-001.jpg:   0%|          | 0.00/254k [00:00<?, ?B/s]

2021-005-M2-002-transformed.jpeg:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

2021-005-M2-005-transformed.jpeg:   0%|          | 0.00/3.00M [00:00<?, ?B/s]

2021-005-M2-007.jpg:   0%|          | 0.00/219k [00:00<?, ?B/s]

tom-h-images-4.zip:   0%|          | 0.00/4.47M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8 [00:00<?, ? examples/s]

##Training

###Logging into Hugging Face and Weights & Biases using API tokens for authentication and experiment tracking.

In [3]:
from huggingface_hub import login
from google.colab import userdata

HF_TOKEN = userdata.get('HF_TOKEN')
login(token=HF_TOKEN)
WANDB_API_KEY = userdata.get('WANDB_API_KEY')
wandb.login(key=WANDB_API_KEY)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: deidaratobi (deidaratobi-national-institute-of-technology-andhra-pradesh) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

###Cloning the Diffusers GitHub repository to access training utilities and scripts.

In [4]:
!pip install git+https://github.com/huggingface/diffusers

  Cloning https://github.com/huggingface/diffusers to /tmp/pip-req-build-f6ct0bm5
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers /tmp/pip-req-build-f6ct0bm5
  Resolved https://github.com/huggingface/diffusers to commit ec3d58286d72e8193b062f2bce7340ddd4c4defb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.34.0.dev0-py3-none-any.whl size=3602897 sha256=2dfef5516c19f2b992dd6725da22ca20b198ba22735460d949636ce474ddb083
  Stored in directory: /tmp/pip-ephem-wheel-cache-cb8463eb/wheels/90/fb/48/a310c271ab42899362ff272062ced42133e5c4c9d0ce77df68
Successfully built diffusers
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.33.1
    Uninstalling diffusers-0.33.1:
      Successfully uninstalled diffusers-0.33.1


In [7]:
!git clone https://github.com/huggingface/diffusers.git
%cd diffusers
%cd examples/dreambooth

Cloning into 'diffusers'...
remote: Enumerating objects: 91589, done.
remote: Counting objects: 100% (241/241), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 91589 (delta 175), reused 127 (delta 125), pack-reused 91348 (from 2)
Receiving objects: 100% (91589/91589), 67.40 MiB | 25.34 MiB/s, done.
Resolving deltas: 100% (67369/67369), done.
/content/diffusers
/content/diffusers/examples/dreambooth


###Training the Stable Diffusion model using LoRA to apply lightweight, parameter-efficient fine-tuning.

In [8]:
!accelerate launch train_dreambooth_lora.py \
  --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5" \
  --instance_data_dir="/content/training_data/images" \
  --output_dir="/content/lora_tom_model" \
  --instance_prompt="a photo of sks person" \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=1 \
  --checkpointing_steps=100 \
  --learning_rate=1e-4 \
  --report_to="wandb" \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=500 \
  --validation_prompt="A photo of sks person cooking pasta." \
  --validation_epochs=50 \
  --seed=42 \
  --push_to_hub

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-05-03 09:16:29.803255: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746263790.045137    1955 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746263790.103176    1955 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-03 09:16:30.562207: I tensorflow/core/platform

##Inference

###Loading the fine-tuned checkpoint

In [9]:
from diffusers import DiffusionPipeline
pipe = DiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5").to("cuda")
pipe.load_lora_weights("/content/lora_tom_model/checkpoint-500")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

No LoRA keys associated to CLIPTextModel found with the prefix='text_encoder'. This is safe to ignore if LoRA state dict didn't originally have any CLIPTextModel related params. You can also try specifying `prefix=None` to resolve the warning. Otherwise, open an issue if you think it's unexpected: https://github.com/huggingface/diffusers/issues/new


In [11]:
%mkdir /content/output

In [13]:
%cd /content/output

/content/output


###Running inference to generate new images.

In [17]:
# Define prompts for image generation
prompts = [
    "Watercolor style image of sks person in a spacesuit, face clearly visible",
    "Watercolor style image of sks person riding horse",
    "Watercolor potrait of sks person playing cricket"
]

# Generate and save images
for i, prompt in enumerate(prompts):
    image = pipe(prompt, num_inference_steps=30, guidance_scale=7.5).images[0]
    image.save(f"generated_scene_{i+1}.png")

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

In [48]:
# move training_data and output folders into mydrive/realityAI

from google.colab import drive
drive.mount('/content/drive')

!mkdir -p /content/drive/MyDrive/realityAI
!mv /content/training_data /content/drive/MyDrive/realityAI
!mv /content/output /content/drive/MyDrive/realityAI


Mounted at /content/drive


In [15]:
%cd /content

/content


In [46]:
prompts = [
    "Upper-body watercolor portrait of sks person in a spacesuit, highly detailed sks person face in focus, soft lighting",
    "watercolor-style painting of sks person, riding a horse.Accurate sks person face description, upper body visible, horse partially shown behind",
    "Close-up watercolor portrait of sks person playing cricket, focus on face with helmet or cap"
]
# Generate and save images
for i, prompt in enumerate(prompts):
    image = pipe(prompt, num_inference_steps=30, guidance_scale=7.5).images[0]
    image.save(f"generated_scene_{i+1}.png")

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

In [47]:
# !mv /content/generated_scene_3.png /content/output/
# !mv /content/generated_scene_1.png /content/output/
!mv /content/generated_scene_2.png /content/output/